Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf), Chapter 8
  - _**[Gradient Boosting Explained](https://www.gormanalysis.com/blog/gradient-boosting-explained/)**_ — Ben Gorman
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) — Alex Rogozhnikov
  - [How to explain gradient boosting](https://explained.ai/gradient-boosting/) — Terence Parr & Jeremy Howard

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re

df = pd.read_csv("https://raw.githubusercontent.com/ekoly/DS-Unit-2-Applied-Modeling/master/data/ufc/data.csv")

In [2]:
r_fighter_cols = [col for col in df.columns if col.startswith("R_")]
b_fighter_cols = [col for col in df.columns if col.startswith("B_")]

other_cols = [col for col in df.columns if col not in r_fighter_cols and col not in b_fighter_cols]

def extractFighters(df, fighter_cols, opp_cols, fighter_prefix, opp_prefix, fighter_color):

    fighters_df = df[fighter_cols + other_cols].copy()
    fighters_df.columns = [col.replace(fighter_prefix, "") for col in fighters_df.columns]
    fighters_df["is_winner"] = fighters_df["Winner"] == fighter_color

    opp_df = df[opp_cols].copy()
    opp_df.columns = [re.sub(r"^" + opp_prefix, "opponent_", col) for col in opp_df.columns]

    fighters_df = pd.concat([fighters_df, opp_df], axis=1)
    
    return fighters_df

r_fighters_df = extractFighters(df, r_fighter_cols, b_fighter_cols, "R_", "B_", "Red")
b_fighters_df = extractFighters(df, b_fighter_cols, r_fighter_cols, "B_", "R_", "Blue")

fighters_df = pd.concat([r_fighters_df, b_fighters_df], axis=0)

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
fighters_df.sort_index(axis=1, inplace=True)

In [3]:
fighters_df.head()

,Height_cms,Reach_cms,Referee,Stance,Weight_lbs,Winner,age,avg_BODY_att,avg_BODY_landed,avg_CLINCH_att,...,total_time_fought(seconds),total_title_bouts,weight_class,win_by_Decision_Majority,win_by_Decision_Split,win_by_Decision_Unanimous,win_by_KO/TKO,win_by_Submission,win_by_TKO_Doctor_Stoppage,wins
0,162.56,162.56,Marc Goddard,Orthodox,135.0,Red,32.0,21.900000,16.400000,17.000000,...,742.60,3.0,Bantamweight,0.0,2.0,4.0,2.0,0.0,0.0,8.0
1,165.10,167.64,Robert Madrigal,Southpaw,125.0,Red,31.0,12.000000,7.714286,9.285714,...,1062.00,2.0,Women's Flyweight,0.0,1.0,2.0,0.0,2.0,0.0,5.0
2,180.34,193.04,Dan Miragliotta,Orthodox,155.0,Red,35.0,13.866667,8.666667,2.866667,...,604.40,2.0,Lightweight,0.0,1.0,3.0,3.0,6.0,1.0,14.0
3,162.56,172.72,Kevin MacDonald,Orthodox,135.0,Blue,29.0,18.250000,10.250000,5.875000,...,690.25,0.0,Bantamweight,0.0,1.0,4.0,1.0,0.0,0.0,6.0
4,187.96,190.50,Dan Miragliotta,Southpaw,264.0,Blue,26.0,7.750000,6.750000,11.000000,...,440.75,0.0,Heavyweight,0.0,0.0,1.0,2.0,0.0,0.0,3.0


In [ ]:
fighers_df['is_winner'].value_counts()